In [7]:
import os
os.listdir()


['app.py', 'app_practice.ipynb', 'backend.py', 'prompts.py', '__pycache__']

In [ ]:
import gradio as gr
from prompts import QUOTE_PROMPT
from backend import (
    generate_study_plan,
    generate_quote,
    generate_quick_insights,
    extract_next_task,
    load_study_plan,
    load_habit_data,
    log_study_session,
    generate_weekly_summary
)



# -------------------------
# Dashboard placeholder loader
# -------------------------
from backend import (
    generate_quick_insights,
    extract_next_task,
    load_study_plan,
    load_habit_data
)


def load_dashboard():
        quote_html = "<div class='quote-bubble'> Quote will appear here...</div>"
        hours_html = "<div class='kpi-box'>Hours Studied: --</div>"
        streak_html = "<div class='kpi-box'>Study Streak: --</div>"
        task_html = "<div class='next-task'> Your next task will appear here...</div>"

        return quote_html, hours_html, streak_html, task_html
# -------------------------
# App Layout
# -------------------------
def load_css(theme):
    filename = "assets/dark.css" if theme == "Dark" else "assets/light.css"
    with open(filename, "r") as f:
        return f.read()

dark_css = load_css("Dark")
light_css = load_css("Light")
def switch_theme(choice):
                return light_css if choice == "Light" else dark_css


with gr.Blocks(css=dark_css) as demo:
    theme_state = gr.State("Dark")
    with gr.Tabs():
        
 
        # -----------------------------------------
        # HOME TAB
        # -----------------------------------------
        with gr.Tab("Home"):

            # Header
            gr.Markdown("<div class='sense-header'> SenseFlow Dashboard</div>")

            # Quote bubble
            quote_box = gr.Markdown("<div class='quote-bubble'> Quote loading...</div>")
            theme_state = gr.State("dark")
           
            theme_toggle = gr.Radio(
            ["Dark", "Light"],
            value="Dark",
            label="Theme",
            elem_classes="theme-toggle"
        )
            
          


            # KPI + Donut layout
            with gr.Row():

                # LEFT SIDE KPIs
                with gr.Column(scale=1):
                    kpi_hours = gr.Markdown("<div class='kpi-box'>Hours Studied: --</div>")
                    kpi_streak = gr.Markdown("<div class='kpi-box'>Study Streak: --</div>")

                # RIGHT SIDE DONUT
                with gr.Column(scale=2):
                    gr.Markdown(
                        "<div style='color:white; font-size:20px; margin-bottom:10px;'>Progress Donut</div>"
                    )
                    donut_placeholder = gr.Markdown(
                        "<div style='color:white;'> Donut chart will appear here (Tasks Completed vs Tasks Left)</div>"
                    )

            # NEXT TASK BOX
            next_task_box = gr.Markdown(
                "<div class='next-task'>📌 Your next task is...<br>(Will be populated from study plan)</div>"
            )

            # NAVIGATION BUTTONS
            gr.Markdown("<div style='color:white; font-size:20px; margin-top:25px;'>Navigation</div>")

            with gr.Row():
                planner_btn = gr.Button("📘 Generate Study Plan", elem_classes="nav-btn")
                calendar_btn = gr.Button("📅 Calendar ", elem_classes="nav-btn")
                tracker_btn = gr.Button("📝 Log Studies", elem_classes="nav-btn")
                quote_btn = gr.Button("🔄 Refresh Quote", elem_classes="nav-btn")
                quote_btn.click(
                fn=lambda: f"<div class='quote-bubble'>{generate_quote()}</div>",
                inputs=None,
                outputs=quote_box
)
       
        
        # -----------------------------------------
        # OTHER TABS (unchanged)
        # -----------------------------------------
        with gr.Tab("Study Planner"):

            gr.Markdown("<div class='planner-title'>STUDY PLANNER</div>")

            # ---- Add Task UI ----
            new_task = gr.Textbox(label="Add a Task")
            add_task_btn = gr.Button("➕ Add Task", elem_classes="nav-btn")

            # This holds the tasks internally
            task_list_state = gr.State([])

            # Where tasks will be shown on screen
            task_list_display = gr.Markdown("<div class='task-list'>No tasks added yet.</div>")

            # ---- Function to update tasks ----
            def add_task(task, task_list):
                if not task.strip():
                    return task_list, "<div class='task-list'>Enter a task first.</div>"

                task_list.append(task)
                html = "<div class='task-list'><ul>"
                for t in task_list:
                    html += f"<li>{t}</li>"
                html += "</ul></div>"

                return task_list, html

            add_task_btn.click(
                fn=add_task,
                inputs=[new_task, task_list_state],
                outputs=[task_list_state, task_list_display]
            )

            # ---- Study plan options ----
            time_input = gr.Slider(1, 8, label="Available Hours")
            difficulty = gr.Dropdown(["Easy", "Medium", "Hard"])
            style = gr.Dropdown(["Pomodoro", "Deep Work", "Short Sessions"])

            generate_btn = gr.Button("Generate Plan", elem_classes="nav-btn")
            plan_output = gr.Markdown()

            # Function to combine tasks for the LLM (later)
            def prepare_plan(task_list, hours, diff, style):
                tasks_joined = "\n".join(task_list)
                return generate_study_plan(tasks_joined, hours, diff, style)

            generate_btn.click(
                fn=prepare_plan,
                inputs=[task_list_state, time_input, difficulty, style],
                outputs=plan_output
            )



        with gr.Tab("Habit Tracker"):
            hours_input = gr.Slider(0, 6, label="Hours Studied")
            tasks_completed = gr.Slider(0, 10, label="Tasks Completed")
            log_btn = gr.Button("Log Session")
            log_output = gr.Markdown()
            log_btn.click(
            fn=log_study_session,
            inputs=[hours_input, tasks_completed],
            outputs=log_output
)

        with gr.Tab("Calender"):
           gr.Markdown("<div class = 'cal-title'>Daily Planner</div>")

        with gr.Tab("Motivation"):

            # ---- Page Title ----
            gr.Markdown("<div class='motivation-title'>STUDY MOTIVATION</div>")

            # ---- Quote Placeholder ----
            motivation_quote = gr.Markdown(
                "<div class='motivation-quote'>Your motivational quote will appear here...</div>"
            )

            refresh_quote_btn = gr.Button("🔄 Refresh Quote", elem_classes="nav-btn")

            gr.Markdown("<hr style='border:1px solid #444; margin-top:20px;'>")

            # ---- Deadline Input Section ----
            gr.Markdown("<div class='deadline-title'>ADD A DEADLINE</div>")

            deadline_name = gr.Textbox(label="Deadline Name (e.g., Maths Exam)")
            deadline_date = gr.Textbox(label="Deadline Date (YYYY-MM-DD HH:MM)")

            add_deadline_btn = gr.Button("➕ Add Deadline", elem_classes="nav-btn")

            # ---- Deadline List Placeholder ----
            deadlines_list = gr.Markdown(
                "<div class='deadline-list'>Your deadlines will appear here...</div>"
            )

            # Temporary UI-only function (no real logic)
            add_deadline_btn.click(
                fn=lambda name, date: f"<div class='deadline-list'>Added: {name} — {date}</div>",
                inputs=[deadline_name, deadline_date],
                outputs=deadlines_list
            )

    
    theme_toggle.change(
            fn=switch_theme,
            inputs=theme_toggle,
            outputs=demo,
        )


demo.launch()


C:\Users\emman\AppData\Local\Temp\ipykernel_70844\551066773.py:48: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=dark_css) as demo:


* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\emman\AppData\Local\Programs\Python\Python314\Lib\site-packages\gradio\queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\emman\AppData\Local\Programs\Python\Python314\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\emman\AppData\Local\Programs\Python\Python314\Lib\site-packages\gradio\blocks.py", line 2136, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\emman\AppData\Local\Programs\Python\Python314\Lib\site-packages\gradio\blocks.py", line 1908, in postprocess_data
    raise InvalidCompo